<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/06_nearest_neighbors_2_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k近傍法（続）

## 6.1 準備

### 6.1.1 インポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

### 6.1.2 データファイルのアップロード

* `lifesat_extended.csv`をアップロードする。

In [ ]:
from google.colab import files
files.upload()

Saving lifesat_extended.csv to lifesat_extended.csv


{'lifesat_extended.csv': b'Country,Air pollution,Assault rate,Consultation on rule-making,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Homicide rate,Household net adjusted disposable income,Household net financial wealth,Housing expenditure,Job security,Life expectancy,Life satisfaction,Long-term unemployment rate,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education,GDP per capita\nRussia,15.0,3.8,2.5,15.1,94.0,0.16,69.0,12.8,19292.0,3412.0,11.0,4.0,70.2,6.0,1.7,20885.0,90.0,0.9,37.0,481.0,14.97,65.0,56.0,16.0,9054.914\nTurkey,35.0,5.0,5.5,12.7,34.0,40.86,50.0,1.2,14095.0,3251.0,21.0,8.1,74.6,5.6,2.37,16919.0,86.0,1.1,68.0,462.0,13.42,88.0,62.0,16.4,9437.372\nHungary,15.0,3.6,7.9,4.8,82.0,3.19,58.0,1.3,15442.0,13277.0,20.0,5.7,75.2,4.9,5.1,20948.0,87.0,1.1,57.0,487.0,15.04,62.0,77.0,17.6,1223

### 6.1.3 データファイルの読み込み
* 今回は、ファイルを読み込むときに、国名をインデックスに設定してしまう。
 * index_col引数を使っている。

In [ ]:
df = pd.read_csv('lifesat_extended.csv', index_col=0)

In [ ]:
df

,Air pollution,Assault rate,Consultation on rule-making,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Homicide rate,Household net adjusted disposable income,Household net financial wealth,...,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education,GDP per capita
Country,,,,,,,,,,,,,,,,,,,,,
Russia,15.0,3.8,2.5,15.1,94.0,0.16,69.0,12.8,19292.0,3412.0,...,20885.0,90.0,0.9,37.0,481.0,14.97,65.0,56.0,16.0,9054.914
Turkey,35.0,5.0,5.5,12.7,34.0,40.86,50.0,1.2,14095.0,3251.0,...,16919.0,86.0,1.1,68.0,462.0,13.42,88.0,62.0,16.4,9437.372
Hungary,15.0,3.6,7.9,4.8,82.0,3.19,58.0,1.3,15442.0,13277.0,...,20948.0,87.0,1.1,57.0,487.0,15.04,62.0,77.0,17.6,12239.894
Poland,33.0,1.4,10.8,3.2,90.0,7.41,60.0,0.9,17852.0,10919.0,...,22655.0,91.0,1.1,58.0,521.0,14.20,55.0,79.0,18.4,12495.334
Slovak Republic,13.0,3.0,6.6,0.6,92.0,7.02,60.0,1.2,17503.0,8663.0,...,20307.0,90.0,1.1,66.0,472.0,14.99,59.0,81.0,16.3,15991.736
Estonia,9.0,5.5,3.3,8.1,90.0,3.30,68.0,4.8,15167.0,7680.0,...,18944.0,89.0,1.5,54.0,526.0,14.90,64.0,79.0,17.5,17288.083
Greece,27.0,3.7,6.5,0.7,68.0,6.16,49.0,1.6,18575.0,14579.0,...,25503.0,83.0,1.2,74.0,466.0,14.91,64.0,69.0,18.6,18064.288
Portugal,18.0,5.7,6.5,0.9,38.0,9.62,61.0,1.1,20086.0,31245.0,...,23688.0,86.0,1.6,46.0,488.0,14.95,58.0,86.0,17.6,19121.592
Slovenia,26.0,3.9,10.3,0.5,85.0,5.63,63.0,0.4,19326.0,18465.0,...,32037.0,90.0,1.5,65.0,499.0,14.62,52.0,88.0,18.4,20732.482


* 日本をtest setとして除外し、残りのデータ集合を使う。
 * 日本の生活満足度を予測することを、最終的な目標としたのだった。

In [ ]:
df_test = df.loc[['Japan']]
df = df.drop(['Japan'])

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 5))
df.plot(kind='scatter', x='GDP per capita', y='Life satisfaction', ax=ax[0])
df.plot(kind='scatter', x='Employment rate', y='Life satisfaction', ax=ax[1]);

In [ ]:
X = df[['GDP per capita', 'Employment rate']]
y = df['Life satisfaction']

In [ ]:
X

## 6.2 二つの特徴量を同時に使う
* 前回は、一人当たりのGDPと、雇用率を、別々に使った。
* 今回は、これら二つの特徴量を、同時に使いたい。
* つまり、(一人当たりのGDP, 雇用率)という2次元ベクトルを使って、生活満足度を予測したい。

### 6.2.1 演習

* 演習問題6-1: 韓国とイタリアの距離を、一人当たりのGDPと雇用率を同時に使って計算したい。しかし、下に示す距離の計算方法には、問題がある。どのような問題があるか。

In [ ]:
print(np.linalg.norm(X.loc['Korea'] - X.loc['Italy']))

2671.395978782627


* 演習問題6-2: 上で見つけた問題を解決した上で、あらためて、韓国とイタリアの距離を求めてみよう。

## 6.3 訓練データ/検証データ/テストデータ


### 6.3.1 テストデータ
* 最終的にそれについて予測を行なうことで、手法の評価をおこないたいデータを、テストデータと呼ぶ。
* よって、今回は、日本のデータが、テストデータになる。
 * 普通は、テストデータは今回のように一つだけではなく、複数、用意する。
 * 今回は、データ数そのものが29個と少ないため、テストデータは一つだけにしている。

### 6.3.2 検証データ

* 最適な近傍の個数kを、どうやって求めたらいいだろうか。
* kの値を変えながら、日本の生活満足度を予測し、予測誤差を見ながらkを決めるのでは、カンニングしていることになる。
 * テストデータでの評価は、最終評価なので、最後に一度行うだけ。
 * つまり、テストデータについては、答えが分かっていないと想定する。
* 日本の生活満足度を予測する前に、どのようにしてkを決めればいいだろうか。


* そこで、テストデータ以外のデータを使って、予測問題を解く。

* テストデータ以外のデータの中から、国を一つ選んで（例えば韓国）、その国について予測をしてみる。
* k近傍法の場合のkのように、チューニングしなければならないパラメータを決めるために使うデータを、検証データと呼ぶ。
 * 本当にそれについて予測をしたいデータは、テストデータ。
 * 検証(validation)のためにそれについて予測をおこなうデータが、検証データ。
* 検証データについて予測を行うことで、kの値を変えたとき予測性能がどう変わるか、調べる。
 * 検証データも、普通は、複数個用意する。
 * 今回は、データ数そのものが少ないため、検証データも一つだけにしている。


### 6.3.3 訓練データ
* 特定のインスタンスについて予測を実行するために使うデータを、訓練データと呼ぶ。
* k近傍法では、そのなかから、特定の国の近傍をk個見つけてくるデータが、訓練データになる。

## 6.4 検証データを使った最適なkの決定
* ここでは、韓国を検証データとして使う。
 * 日本はテストデータなので、最初に除外しておいた。
* そして、韓国の生活満足度を予測し、最も良い予測値を与えるkがいくらか、調べる。

### 6.4.1 演習

* 演習問題6-3: 韓国の生活満足度を予測し、最も良い予測値を与えるkを調べよう。

In [ ]:
# スケーリングのための最大値、最小値を求める
country = 'Korea'
min_val = X.drop(country).min() # ここでdropしなければ授業での説明のときの方法になる
max_val = X.drop(country).max() #
print(f"min={min_val.values} max={max_val.values}")

min=[9054.914   49.   ] max=[55805.204    82.   ]


In [ ]:
# スケーリングして距離を計算する
X_scaled = (X - min_val) / (max_val - min_val)
distance = np.sqrt(((X_scaled - X_scaled.loc[country]) ** 2).sum(axis=1))

In [ ]:
# 近い順にkヶ国の生活満足度の平均値を予測値としたときの予測誤差を求める
sorted_countries = distance.sort_values().drop(country).index
errors = []
for k in range(1, len(sorted_countries)):
  error = abs(y[sorted_countries[:k]].mean() - y[country])
  errors.append(error)
print(errors)

[0.09999999999999964, 0.39999999999999947, 0.2666666666666675, 0.3750000000000009, 0.4400000000000004, 0.3333333333333339, 0.31428571428571495, 0.3125000000000009, 0.40000000000000036, 0.3600000000000003, 0.46363636363636473, 0.5583333333333345, 0.44615384615384723, 0.5000000000000018, 0.5333333333333341, 0.5125000000000002, 0.5705882352941174, 0.6000000000000005, 0.647368421052632, 0.5650000000000004, 0.5952380952380958, 0.6363636363636376, 0.6000000000000005, 0.6333333333333337, 0.6760000000000002, 0.703846153846154]


In [ ]:
print(f"{country} | best k={np.array(errors).argmin() + 1}")

Korea | best k=1


## 6.5 ‎leave-one-out交差検証 (1)
* 上では、韓国を検証データとして使った。
* しかし、韓国以外の国を検証データとしても構わないはずである。
* そこで、テストデータである日本以外の国について、その一つ一つを検証データとした場合の、それぞれの最適なkの値を求めてみる。

### 6.5.1 演習

* 演習問題6-4: 上で韓国について行ったことと同じことを、他の国についても実行し、最適なkの値がどのくらい違ってくるものか、見てみよう。

In [ ]:
# それぞれのkの値での予測誤差を求める関数を定義する
def pred_error(country):
  min_val = X.drop(country).min()
  max_val = X.drop(country).max()
  X_scaled = (X - min_val) / (max_val - min_val)
  distance = np.sqrt(((X_scaled - X_scaled.loc[country]) ** 2).sum(axis=1))
  sorted_countries = distance.sort_values().drop(country).index
  errors = []
  for k in range(1, len(sorted_countries)):
    error = abs(y[sorted_countries[:k]].mean() - y[country])
    errors.append(error)
  return np.array(errors)

In [ ]:
country = 'Korea'
print(f"{country} | best k={pred_error(country).argmin() + 1}")

Korea | best k=1


In [ ]:
for country in X.index:
  print(f"{country} | best k={pred_error(country).argmin() + 1}")

Russia | best k=15
Turkey | best k=8
Hungary | best k=8
Poland | best k=12
Slovak Republic | best k=16
Estonia | best k=3
Greece | best k=1
Portugal | best k=4
Slovenia | best k=7
Spain | best k=26
Korea | best k=1
Italy | best k=5
Israel | best k=4
New Zealand | best k=3
France | best k=26
Belgium | best k=18
Germany | best k=17
Finland | best k=7
Canada | best k=10
Netherlands | best k=11
Austria | best k=19
United Kingdom | best k=21
Sweden | best k=8
Iceland | best k=3
Australia | best k=2
Ireland | best k=4
Denmark | best k=1
United States | best k=9


* 最適なkを、どのようにして決めればいいだろうか？
 * 例えば、各国について得られたkの平均をとることが考えられるが、これは良い方法と言えるだろうか？

## 6.6 ‎leave-one-out交差検証 (2)
* 上では、テストデータである日本を除く国を、一つずつ順番に検証データとして、最適なkを求めた。
* しかし、こうして国ごとに求められたkを、どうやってまとめあげるのか、疑問が残った。
 * それらkの平均をとる、という方法は、何かおかしい感じがする。
 * 最頻値の方が、まだましであろう。
* 最適なkを決める、もっと良い方法はないだろうか？

### 6.6.1 演習

* 演習問題6-5: 韓国を検証データとする。つまり、韓国について予測を行なう。k=1から27まで変えたとき、それぞれ予測誤差がいくらになるか、求めてみよう。

In [ ]:
pred_error('Korea')

array([0.1       , 0.4       , 0.26666667, 0.375     , 0.44      ,
       0.33333333, 0.31428571, 0.3125    , 0.4       , 0.36      ,
       0.46363636, 0.55833333, 0.44615385, 0.5       , 0.53333333,
       0.5125    , 0.57058824, 0.6       , 0.64736842, 0.565     ,
       0.5952381 , 0.63636364, 0.6       , 0.63333333, 0.676     ,
       0.70384615])

* 演習問題6-6: 縦が国名、横がkの値の表を、データフレームとして作り、日本を除く28カ国の国のひとつひとつを検証データとしたとき、各々のkの値で予測誤差がいくらになるかで埋めてみよう。

In [ ]:
error_dict = {}
for country in X.index:
  error_dict[country] = pred_error(country)
error_df = pd.DataFrame.from_dict(error_dict, orient='index')
error_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
Russia,0.4,0.30,3.000000e-01,0.200,3.400000e-01,0.466667,0.428571,0.3125,0.122222,0.15,...,1.411765e-01,0.205556,0.236842,0.270,0.319048,0.359091,0.400000,0.445833,0.468,0.507692
Turkey,0.8,0.75,4.333333e-01,0.200,2.600000e-01,0.066667,0.042857,0.0125,0.033333,0.03,...,5.647059e-01,0.611111,0.642105,0.695,0.723810,0.768182,0.808696,0.841667,0.872,0.911538
Hungary,0.9,1.05,7.666667e-01,0.775,7.600000e-01,0.900000,0.757143,0.7500,0.788889,0.80,...,1.305882e+00,1.338889,1.394737,1.425,1.471429,1.500000,1.539130,1.570833,1.612,1.638462
Poland,0.9,0.30,4.333333e-01,0.350,3.200000e-01,0.233333,0.228571,0.1125,0.100000,0.19,...,3.529412e-01,0.388889,0.447368,0.480,0.528571,0.559091,0.600000,0.633333,0.676,0.703846
Slovak Republic,1.0,0.65,8.333333e-01,0.725,5.000000e-01,0.500000,0.471429,0.4250,0.388889,0.40,...,3.529412e-02,0.072222,0.131579,0.165,0.214286,0.245455,0.286957,0.320833,0.364,0.392308
Estonia,0.1,0.25,8.881784e-16,0.125,1.400000e-01,0.150000,0.028571,0.2500,0.322222,0.46,...,7.117647e-01,0.738889,0.657895,0.690,0.738095,0.777273,0.817391,0.862500,0.884,0.911538
Greece,0.8,1.25,8.666667e-01,0.950,1.020000e+00,1.016667,0.914286,0.9125,0.922222,0.91,...,1.411765e+00,1.444444,1.484211,1.535,1.561905,1.604545,1.639130,1.675000,1.704,1.742308
Portugal,0.6,0.80,7.666667e-01,0.525,5.600000e-01,0.700000,0.671429,0.7000,0.722222,0.62,...,1.094118e+00,1.127778,1.184211,1.215,1.261905,1.290909,1.330435,1.362500,1.404,1.430769
Slovenia,0.6,0.10,3.333333e-02,0.050,2.000000e-02,0.150000,0.014286,0.0250,0.055556,0.22,...,4.588235e-01,0.494444,0.552632,0.585,0.633333,0.663636,0.704348,0.737500,0.780,0.807692
Spain,0.5,0.95,9.000000e-01,0.775,7.600000e-01,0.916667,1.014286,0.9750,0.866667,0.74,...,3.882353e-01,0.350000,0.305263,0.250,0.219048,0.172727,0.134783,0.095833,0.064,0.023077


* 演習問題6-7: 上で作った表で、それぞれのkの値について、予測誤差の平均値を求めてみよう。

In [ ]:
error_df.mean()

0     0.453571
1     0.392857
2     0.361905
3     0.313393
4     0.319286
5     0.330952
6     0.314286
7     0.309375
8     0.305952
9     0.303571
10    0.312013
11    0.322619
12    0.323352
13    0.333929
14    0.357857
15    0.394643
16    0.427941
17    0.454365
18    0.486278
19    0.516250
20    0.553741
21    0.582630
22    0.609472
23    0.647024
24    0.689143
25    0.731181
dtype: float64

* 演習問題6-8: 上で求めた予測誤差の平均値を、最も小さくするkの値は、いくらになるか。

In [ ]:
k = error_df.mean().argmin() + 1
print(k)

10


* 演習問題6-9: こうして求めた最適なkを使って、最終的に、日本の生活満足度の予測をしてみよう。

In [ ]:
min_val = X.min() # 日本だけが除外されている
max_val = X.max()
X_scaled = (X - min_val) / (max_val - min_val)

In [ ]:
X_scaled

,GDP per capita,Employment rate
Country,,
Russia,0.000000,0.606061
Turkey,0.008181,0.030303
Hungary,0.068127,0.272727
Poland,0.073591,0.333333
Slovak Republic,0.148380,0.333333
Estonia,0.176109,0.575758
Greece,0.192713,0.000000
Portugal,0.215329,0.363636
Slovenia,0.249786,0.424242


In [ ]:
X_test = df_test[['GDP per capita', 'Employment rate']]
X_test_scaled = (X_test - min_val) / (max_val - min_val)
X_test_scaled

,GDP per capita,Employment rate
Country,,
Japan,0.501187,0.69697


In [ ]:
X_scaled - X_test_scaled.values

,GDP per capita,Employment rate
Country,,
Russia,-0.501187,-0.090909
Turkey,-0.493006,-0.666667
Hungary,-0.433059,-0.424242
Poland,-0.427595,-0.363636
Slovak Republic,-0.352807,-0.363636
Estonia,-0.325077,-0.121212
Greece,-0.308474,-0.696970
Portugal,-0.285858,-0.333333
Slovenia,-0.251401,-0.272727


In [ ]:
distance = np.sqrt(((X_scaled - X_test_scaled.values) ** 2).sum(axis=1))
sorted_countries = distance.sort_values().index

In [ ]:
distance[sorted_countries]

Country
New Zealand        0.102125
Israel             0.163382
Germany            0.184556
Finland            0.222390
Canada             0.232007
Austria            0.240394
United Kingdom     0.243287
Netherlands        0.245409
France             0.266630
Korea              0.267535
Belgium            0.344096
Estonia            0.346940
Slovenia           0.370921
Sweden             0.376685
Australia          0.395213
Denmark            0.420953
Portugal           0.439120
Italy              0.488074
Iceland            0.496198
Spain              0.505108
Slovak Republic    0.506660
Russia             0.509365
United States      0.521317
Ireland            0.543202
Poland             0.561310
Hungary            0.606236
Greece             0.762183
Turkey             0.829156
dtype: float64

In [ ]:
sorted_countries

Index(['New Zealand', 'Israel', 'Germany', 'Finland', 'Canada', 'Austria',
       'United Kingdom', 'Netherlands', 'France', 'Korea', 'Belgium',
       'Estonia', 'Slovenia', 'Sweden', 'Australia', 'Denmark', 'Portugal',
       'Italy', 'Iceland', 'Spain', 'Slovak Republic', 'Russia',
       'United States', 'Ireland', 'Poland', 'Hungary', 'Greece', 'Turkey'],
      dtype='object', name='Country')

In [ ]:
y_test = df_test['Life satisfaction']
y_test

Country
Japan    5.9
Name: Life satisfaction, dtype: float64

In [ ]:
y[sorted_countries[:k]].mean()

6.970000000000001

In [ ]:
abs(y[sorted_countries[:k]].mean() - y_test)

Country
Japan    1.07
Name: Life satisfaction, dtype: float64

## スケーリングの際に検証データを含めた場合
* こちらのほうが授業での説明の通りの方法。

In [ ]:
# それぞれのkの値での予測誤差を求める関数を定義する
def pred_error_2(country):
  min_val = X.min()
  max_val = X.max()
  X_scaled = (X - min_val) / (max_val - min_val)
  distance = np.sqrt(((X_scaled - X_scaled.loc[country]) ** 2).sum(axis=1))
  sorted_countries = distance.sort_values().drop(country).index
  errors = []
  for k in range(1, len(sorted_countries)):
    error = abs(y[sorted_countries[:k]].mean() - y[country])
    errors.append(error)
  return np.array(errors)

In [ ]:
country = 'Korea'
print(f"{country} | best k={pred_error_2(country).argmin() + 1}")

Korea | best k=1


In [ ]:
error_dict = {}
for country in X.index:
  error_dict[country] = pred_error_2(country)
error_df_2 = pd.DataFrame.from_dict(error_dict, orient='index')
error_df_2

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
Russia,0.4,0.30,3.000000e-01,0.200,3.400000e-01,0.466667,0.428571,0.3125,0.122222,0.15,...,1.411765e-01,0.205556,0.236842,0.270,0.319048,0.359091,0.400000,0.445833,0.468,0.507692
Turkey,0.8,0.75,4.333333e-01,0.200,2.600000e-01,0.066667,0.042857,0.0125,0.033333,0.03,...,5.647059e-01,0.611111,0.642105,0.695,0.723810,0.768182,0.808696,0.841667,0.872,0.911538
Hungary,0.9,1.05,7.666667e-01,0.775,7.600000e-01,0.900000,0.757143,0.7500,0.788889,0.80,...,1.305882e+00,1.338889,1.394737,1.425,1.471429,1.500000,1.539130,1.570833,1.612,1.638462
Poland,0.9,0.30,4.333333e-01,0.350,3.200000e-01,0.233333,0.228571,0.1125,0.100000,0.19,...,3.529412e-01,0.388889,0.447368,0.480,0.528571,0.559091,0.600000,0.633333,0.676,0.703846
Slovak Republic,1.0,0.65,8.333333e-01,0.725,5.000000e-01,0.500000,0.471429,0.4250,0.388889,0.40,...,3.529412e-02,0.072222,0.131579,0.165,0.214286,0.245455,0.286957,0.320833,0.364,0.392308
Estonia,0.1,0.25,8.881784e-16,0.125,1.400000e-01,0.150000,0.028571,0.2500,0.322222,0.46,...,7.117647e-01,0.738889,0.657895,0.690,0.738095,0.777273,0.817391,0.862500,0.884,0.911538
Greece,0.8,1.25,8.666667e-01,0.950,1.020000e+00,1.016667,0.914286,0.9125,0.922222,0.91,...,1.411765e+00,1.444444,1.484211,1.535,1.561905,1.604545,1.639130,1.675000,1.704,1.742308
Portugal,0.6,0.80,7.666667e-01,0.525,5.600000e-01,0.700000,0.671429,0.7000,0.722222,0.62,...,1.094118e+00,1.127778,1.184211,1.215,1.261905,1.290909,1.330435,1.362500,1.404,1.430769
Slovenia,0.6,0.10,3.333333e-02,0.050,2.000000e-02,0.150000,0.014286,0.0250,0.055556,0.22,...,4.588235e-01,0.494444,0.552632,0.585,0.633333,0.663636,0.704348,0.737500,0.780,0.807692
Spain,0.5,0.95,9.000000e-01,0.775,7.600000e-01,0.916667,1.014286,0.9750,0.866667,0.74,...,3.882353e-01,0.350000,0.305263,0.250,0.219048,0.172727,0.134783,0.095833,0.064,0.023077


In [ ]:
error_df_2.mean()

0     0.453571
1     0.389286
2     0.361905
3     0.313393
4     0.320000
5     0.329167
6     0.314286
7     0.309375
8     0.305952
9     0.303571
10    0.312013
11    0.322619
12    0.323626
13    0.334184
14    0.357857
15    0.394643
16    0.427941
17    0.454365
18    0.486278
19    0.516250
20    0.552891
21    0.581981
22    0.609472
23    0.646875
24    0.689143
25    0.731181
dtype: float64

* さきほどのやり方との差
 * それほどない感じ。

In [ ]:
(error_df_2.mean() - error_df.mean())

0     0.000000e+00
1    -3.571429e-03
2     0.000000e+00
3     0.000000e+00
4     7.142857e-04
5    -1.785714e-03
6     0.000000e+00
7     0.000000e+00
8     0.000000e+00
9     0.000000e+00
10    0.000000e+00
11    0.000000e+00
12    2.747253e-04
13    2.551020e-04
14    0.000000e+00
15    0.000000e+00
16    0.000000e+00
17    0.000000e+00
18    0.000000e+00
19    0.000000e+00
20   -8.503401e-04
21   -6.493506e-04
22    0.000000e+00
23   -1.488095e-04
24    0.000000e+00
25   -1.110223e-16
dtype: float64

In [ ]:
k = error_df_2.mean().argmin() + 1
print(k)

10


* というわけで答えは同じ。

# 標準化する場合
* まずは標準化する際に検証データを含ませる場合

In [ ]:
def pred_error_std(country):
  X_scaled = (X - X.drop(country).mean()) / X.drop(country).std()
  distance = np.sqrt(((X_scaled - X_scaled.loc[country]) ** 2).sum(axis=1))
  sorted_countries = distance.sort_values().drop(country).index
  errors = []
  for k in range(1, len(sorted_countries)):
    error = abs(y[sorted_countries[:k]].mean() - y[country])
    errors.append(error)
  return np.array(errors)

In [ ]:
error_dict = {}
for country in X.index:
  error_dict[country] = pred_error_std(country)
error_df_std = pd.DataFrame.from_dict(error_dict, orient='index')
error_df_std

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
Russia,0.4,0.35,0.300000,0.450,0.34,4.666667e-01,4.285714e-01,0.2000,1.222222e-01,0.02,...,0.288235,0.316667,0.347368,0.395,0.319048,0.359091,0.400000,0.445833,0.468,0.496154
Turkey,0.8,0.75,0.433333,0.100,0.02,8.333333e-02,8.881784e-16,0.0125,3.333333e-02,0.03,...,0.564706,0.600000,0.657895,0.695,0.723810,0.763636,0.804348,0.841667,0.872,0.911538
Hungary,0.9,1.05,0.766667,0.775,0.94,9.000000e-01,7.571429e-01,0.8000,8.111111e-01,0.80,...,1.305882,1.338889,1.378947,1.430,1.457143,1.500000,1.534783,1.570833,1.600,1.638462
Poland,0.3,0.30,0.433333,0.350,0.14,1.500000e-01,1.285714e-01,0.0875,1.000000e-01,0.07,...,0.352941,0.388889,0.447368,0.480,0.514286,0.559091,0.600000,0.633333,0.664,0.703846
Slovak Republic,0.3,0.65,0.833333,0.725,0.50,4.666667e-01,4.714286e-01,0.4250,3.888889e-01,0.40,...,0.023529,0.072222,0.115789,0.170,0.200000,0.245455,0.286957,0.320833,0.352,0.392308
Estonia,0.4,0.25,0.233333,0.050,0.14,1.500000e-01,3.857143e-01,0.2500,3.222222e-01,0.46,...,0.782353,0.811111,0.857895,0.815,0.852381,0.890909,0.817391,0.862500,0.884,0.911538
Greece,0.8,1.25,1.233333,0.950,1.02,1.016667e+00,9.142857e-01,0.9125,9.222222e-01,1.04,...,1.382353,1.444444,1.494737,1.545,1.571429,1.600000,1.639130,1.675000,1.716,1.742308
Portugal,1.0,0.80,0.766667,0.525,0.56,7.000000e-01,6.714286e-01,0.7000,7.222222e-01,0.79,...,1.082353,1.127778,1.184211,1.215,1.247619,1.290909,1.330435,1.362500,1.392,1.430769
Slovenia,0.6,0.25,0.033333,0.000,0.02,1.500000e-01,1.428571e-02,0.0250,5.555556e-02,0.22,...,0.447059,0.494444,0.552632,0.585,0.633333,0.663636,0.704348,0.737500,0.768,0.807692
Spain,0.5,0.95,0.766667,0.975,0.94,9.000000e-01,8.714286e-01,0.9750,8.222222e-01,0.74,...,0.417647,0.350000,0.289474,0.255,0.219048,0.177273,0.134783,0.095833,0.052,0.023077


In [ ]:
k = error_df_std.mean().argmin() + 1
print(k)

10


In [ ]:
mean_val = X.mean() # 日本だけが除外されている
std_val = X.std()
X_scaled = (X - mean_val) / std_val

In [ ]:
X_test = df_test[['GDP per capita', 'Employment rate']]
X_test_scaled = (X_test - mean_val) / std_val
X_test_scaled

,GDP per capita,Employment rate
Country,,
Japan,-0.062916,0.806175


In [ ]:
distance = np.sqrt(((X_scaled - X_test_scaled.values) ** 2).sum(axis=1))
sorted_countries = distance.sort_values().index
sorted_countries

Index(['New Zealand', 'Germany', 'Israel', 'Canada', 'Finland', 'Austria',
       'United Kingdom', 'Netherlands', 'France', 'Korea', 'Estonia', 'Sweden',
       'Australia', 'Denmark', 'Belgium', 'Slovenia', 'Russia', 'Portugal',
       'United States', 'Iceland', 'Slovak Republic', 'Ireland', 'Poland',
       'Italy', 'Spain', 'Hungary', 'Greece', 'Turkey'],
      dtype='object', name='Country')

In [ ]:
y_test = df_test['Life satisfaction']
y_predict = y[sorted_countries[:k]].mean()
print(y_predict)
print(abs(y_predict - y_test.values[0]))

6.970000000000001
1.0700000000000003


## 標準化の際に検証データを除かない場合

In [ ]:
def pred_error_std_2(country):
  X_scaled = (X - X.mean()) / X.std()
  distance = np.sqrt(((X_scaled - X_scaled.loc[country]) ** 2).sum(axis=1))
  sorted_countries = distance.sort_values().drop(country).index
  errors = []
  for k in range(1, len(sorted_countries)):
    error = abs(y[sorted_countries[:k]].mean() - y[country])
    errors.append(error)
  return np.array(errors)

In [ ]:
error_dict = {}
for country in X.index:
  error_dict[country] = pred_error_std_2(country)
error_df_std_2 = pd.DataFrame.from_dict(error_dict, orient='index')
error_df_std_2

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
Russia,0.4,0.35,0.300000,0.450,0.34,0.316667,4.285714e-01,0.2000,3.333333e-02,0.02,...,0.358824,0.388889,0.436842,0.395,0.319048,0.359091,0.400000,0.445833,0.468,0.496154
Turkey,0.8,0.75,0.433333,0.100,0.02,0.066667,8.881784e-16,0.0125,3.333333e-02,0.03,...,0.564706,0.600000,0.657895,0.695,0.723810,0.763636,0.804348,0.841667,0.872,0.911538
Hungary,0.9,1.05,0.766667,0.775,0.94,0.900000,9.285714e-01,0.8000,8.111111e-01,0.80,...,1.305882,1.338889,1.378947,1.430,1.457143,1.500000,1.534783,1.570833,1.600,1.638462
Poland,0.3,0.30,0.433333,0.350,0.14,0.150000,1.285714e-01,0.0875,5.555556e-02,0.07,...,0.341176,0.388889,0.447368,0.480,0.514286,0.559091,0.600000,0.633333,0.664,0.703846
Slovak Republic,0.3,0.65,0.833333,0.725,0.50,0.466667,4.714286e-01,0.4250,3.888889e-01,0.40,...,0.023529,0.072222,0.115789,0.170,0.200000,0.245455,0.286957,0.320833,0.352,0.392308
Estonia,0.4,0.25,0.233333,0.050,0.14,0.150000,3.857143e-01,0.2500,3.222222e-01,0.46,...,0.782353,0.811111,0.857895,0.895,0.933333,0.890909,0.934783,0.862500,0.884,0.911538
Greece,0.8,1.25,1.233333,0.950,1.02,1.016667,9.142857e-01,0.9125,9.222222e-01,1.04,...,1.382353,1.444444,1.500000,1.530,1.571429,1.600000,1.639130,1.675000,1.716,1.742308
Portugal,1.0,0.80,0.766667,0.525,0.56,0.700000,6.714286e-01,0.7000,7.222222e-01,0.79,...,1.082353,1.127778,1.184211,1.220,1.247619,1.290909,1.330435,1.362500,1.392,1.430769
Slovenia,0.6,0.25,0.033333,0.000,0.02,0.150000,1.428571e-02,0.0250,5.555556e-02,0.22,...,0.447059,0.494444,0.552632,0.585,0.633333,0.663636,0.704348,0.737500,0.768,0.807692
Spain,0.5,0.95,0.766667,0.975,0.94,0.900000,8.714286e-01,0.9750,8.222222e-01,0.74,...,0.417647,0.350000,0.289474,0.255,0.219048,0.177273,0.134783,0.095833,0.052,0.023077


In [ ]:
k = error_df_std_2.mean().argmin() + 1
print(k)

9


In [ ]:
mean_val = X.mean() # 日本だけが除外されている
std_val = X.std()
X_scaled = (X - mean_val) / std_val

X_test = df_test[['GDP per capita', 'Employment rate']]
X_test_scaled = (X_test - mean_val) / std_val

distance = np.sqrt(((X_scaled - X_test_scaled.values) ** 2).sum(axis=1))
sorted_countries = distance.sort_values().index

y_test = df_test['Life satisfaction']
y_predict = y[sorted_countries[:k]].mean()
print(y_predict)
print(abs(y_predict - y_test.values[0]))

7.1
1.1999999999999993
